In [1]:
%%capture
# ignore warnings in this cell
%load_ext autoreload
%autoreload 2

import os
os.environ["USE_NUMBA"] = "1"
import random

from eals.eals import ElementwiseAlternatingLeastSquares, load_model
from eals.util import create_user_items

In [2]:
# Data preparation
user_count = 2000
item_count = 1000
data_count = user_count * 20
new_user_count = 200
new_item_count = 100

user_items = lambda: create_user_items(
    user_count=user_count + new_user_count,
    item_count=item_count + new_item_count,
    data_count=data_count,
)

In [3]:
# Benchmarking fit()

model = ElementwiseAlternatingLeastSquares(max_iter=10)
model.fit(user_items())  # warm up
%time model.fit(user_items())

CPU times: user 962 ms, sys: 61.6 ms, total: 1.02 s
Wall time: 153 ms


In [4]:
# Benchmarking update_user()

# existing users only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count))  # warm up
%timeit model.update_user(random.randrange(user_count))

# new users only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count, user_count + new_user_count))  # warm up
%timeit model.update_user(random.randrange(user_count, user_count + new_user_count))

# mixed users
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_user(random.randrange(user_count + new_user_count))  # warm up
%timeit model.update_user(random.randrange(user_count + new_user_count))

24.5 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.3 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.8 µs ± 234 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
# Benchmarking update_item()

# existing items only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count))  # warm up
%timeit model.update_item(random.randrange(item_count))

# new items only
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count, item_count + new_item_count))  # warm up
%timeit model.update_item(random.randrange(item_count, item_count + new_item_count))

# mixed items
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_item(random.randrange(item_count + new_item_count))  # warm up
%timeit model.update_item(random.randrange(item_count + new_item_count))

31.4 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
29.8 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
31.1 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# Benchmarking update_model()
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.update_model(random.randrange(user_count), random.randrange(item_count))  # warm up
%timeit -n 1000 model.update_model(random.randrange(user_count), random.randrange(item_count))

122 µs ± 2.62 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
# Benchmarking calc_loss() for csr matrices
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model.calc_loss()  # warm up
%timeit model.calc_loss()

3.39 ms ± 26.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# Benchmarking calc_loss() for lil matrices
model = ElementwiseAlternatingLeastSquares()
model.init_data(user_items())
model._convert_data_for_online_training()
model.calc_loss()  # warm up
%timeit model.calc_loss()

13.3 ms ± 53.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
# benchmark calc_loss() for a real data, csr matrix
%run benchmark_calc_loss.py -f 2020-08-24-view0100.json -m csr

Benchmarking calc_loss()
real data (2020-08-24-view0100.json), csr matrix
  load data: (use cache: 2020-08-24-view0100-train_data.npy) 2.4458420276641846 sec
    user_count=1200777, item_count=110547, nnz=37342551
  setup: 3.680230140686035 sec
  elapsed: 4.199114084243774 sec


In [10]:
# benchmark calc_loss() for a real data, lil matrix
%run benchmark_calc_loss.py -f 2020-08-24-view0100.json -m lil

Benchmarking calc_loss()
real data (2020-08-24-view0100.json), lil matrix
  load data: (use cache: 2020-08-24-view0100-train_data.npy) 2.061575174331665 sec
    user_count=1200777, item_count=110547, nnz=37342551
  setup: 31.84101700782776 sec
  elapsed: 13.002599954605103 sec


In [11]:
# Model serializetion benchmark
# 2020-08-24-view0100.json と大体同じサイズのランダムデータ
ui = create_user_items(
    user_count=1200000,
    item_count=110000,
    data_count=37000000
)
model = ElementwiseAlternatingLeastSquares()
model.init_data(ui)
model._convert_data_for_online_training()

print("serialize .json")
%time model.save("model.json", compress=False)
print()
print("deserialize .json")
%time model = load_model("model.json")
print()
print("serialize .json.gz")
%time model.save("model.json.gz", compress=True)
print()
print("deserialize .json.gz")
%time model = load_model("model.json.gz")
print()
print("serialize .joblib without compression")
%time model.save("model0.joblib", compress=0)
print()
print("deserialize .joblib without compression")
%time model = load_model("model0.joblib")
print()
print("serialize .joblib with compression")
%time model.save("model9.joblib", compress=9)
print()
print("deserialize .joblib with compression")
%time model = load_model("model9.joblib")

serialize .json
CPU times: user 6.54 s, sys: 2.65 s, total: 9.19 s
Wall time: 9.79 s

deserialize .json
converting type of user_items to np.float32
CPU times: user 1min 41s, sys: 2min 15s, total: 3min 56s
Wall time: 4min 35s

serialize .json.gz
CPU times: user 6min 50s, sys: 4.75 s, total: 6min 55s
Wall time: 6min 59s

deserialize .json.gz
converting type of user_items to np.float32
CPU times: user 1min 43s, sys: 1min 50s, total: 3min 33s
Wall time: 4min 7s

serialize .joblib without compression
CPU times: user 6.42 s, sys: 2.72 s, total: 9.14 s
Wall time: 11 s

deserialize .joblib without compression
CPU times: user 27.9 s, sys: 14.8 s, total: 42.8 s
Wall time: 46.9 s

serialize .joblib with compression
CPU times: user 18min 4s, sys: 6.45 s, total: 18min 10s
Wall time: 18min 15s

deserialize .joblib with compression
CPU times: user 36.1 s, sys: 13.2 s, total: 49.3 s
Wall time: 51.2 s


In [12]:
!ls -lh model*

-rw-r--r--  1 michihiro.nakamura  staff   2.5G Jul 16 05:26 model.json
-rw-r--r--  1 michihiro.nakamura  staff   988M Jul 16 05:37 model.json.gz
-rw-r--r--  1 michihiro.nakamura  staff   2.5G Jul 16 05:42 model0.joblib
-rw-r--r--  1 michihiro.nakamura  staff   1.3G Jul 16 06:01 model9.joblib


In [13]:
for i in range(1, 9):
    print(f"serialize .joblib with compression level {i}")
    %time model.save(f"model{i}.joblib", compress=i)
    print()
    print(f"deserialize .joblib with compression level {i}")
    %time model = load_model(f"model{i}.joblib")
    print()

SyntaxError: invalid syntax (1272609549.py, line 1)